<a href="https://colab.research.google.com/github/evansonorieru/testing/blob/master/02_pydeequ_bank_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First we need to install pyspark, by doing the download of the relevant tar files

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop2.tgz
!tar xf spark-3.3.2-bin-hadoop2.tgz
!pip install -q findspark

Define the environment features

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop2"

In [ ]:
import findspark
findspark.init()

Install pydeequ

In [ ]:
!pip install pydeequ

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
SPARK_VERSION="3.3.2"

In [ ]:
!wget https://repo1.maven.org/maven2/com/amazon/deequ/deequ/2.0.2-spark-3.3/deequ-2.0.2-spark-3.3.jar

--2023-06-06 15:49:39--  https://repo1.maven.org/maven2/com/amazon/deequ/deequ/2.0.2-spark-3.3/deequ-2.0.2-spark-3.3.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1265888 (1.2M) [application/java-archive]
Saving to: ‘deequ-2.0.2-spark-3.3.jar’

deequ-2.0.2-spark-3 100%[===================>]   1.21M  --.-KB/s    in 0.06s   

2023-06-06 15:49:39 (21.8 MB/s) - ‘deequ-2.0.2-spark-3.3.jar’ saved [1265888/1265888]



In [ ]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars file:///content/deequ-2.0.2-spark-3.3.jar pyspark-shell'

In [ ]:
from pyspark.sql import SparkSession, Row
import pydeequ
spark = (SparkSession
             .builder
             .getOrCreate())

ERROR:logger:Please set env variable SPARK_VERSION


In [ ]:
#from google.colab import files
#files.upload()

In [ ]:
!wget -q https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip
!unzip bank.zip

Archive:  bank.zip
  inflating: bank-full.csv           
  inflating: bank-names.txt          
  inflating: bank.csv                


In [ ]:
df = spark.read.option("header","true").options(inferSchema='True',delimiter=";").csv("bank-full.csv")

In [ ]:
df.show()

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
| 47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
| 33|     unknown|  single|  unknown|     no|      1|     no|  no|unknown|  5|  may

In [ ]:
df.dtypes

[('age', 'int'),
 ('job', 'string'),
 ('marital', 'string'),
 ('education', 'string'),
 ('default', 'string'),
 ('balance', 'int'),
 ('housing', 'string'),
 ('loan', 'string'),
 ('contact', 'string'),
 ('day', 'int'),
 ('month', 'string'),
 ('duration', 'int'),
 ('campaign', 'int'),
 ('pdays', 'int'),
 ('previous', 'int'),
 ('poutcome', 'string'),
 ('y', 'string')]

Metrics Computation

In [ ]:
from pydeequ.analyzers import AnalysisRunner, AnalyzerContext, Completeness,Compliance, Correlation, Size, Completeness, Mean, ApproxCountDistinct, Maximum, Minimum, Entropy

In [ ]:
analysisResult = AnalysisRunner(spark)\
                         .onData(df)\
                         .addAnalyzer(Size())\
                         .addAnalyzer(Completeness("balance"))\
                         .addAnalyzer(ApproxCountDistinct("balance"))\
                         .addAnalyzer(Minimum("balance"))\
                         .addAnalyzer(Mean("balance"))\
                         .addAnalyzer(Maximum("balance"))\
                         .addAnalyzer(Entropy("balance"))\
                         .addAnalyzer(Correlation("age", "balance"))\
                         .run()


In [ ]:
analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show(10,False)

+-----------+-----------+-------------------+-------------------+
|entity     |instance   |name               |value              |
+-----------+-----------+-------------------+-------------------+
|Dataset    |*          |Size               |45211.0            |
|Mutlicolumn|age,balance|Correlation        |0.09778273937134817|
|Column     |balance    |Completeness       |1.0                |
|Column     |balance    |ApproxCountDistinct|7375.0             |
|Column     |balance    |Minimum            |-8019.0            |
|Column     |balance    |Mean               |1362.2720576850766 |
|Column     |balance    |Maximum            |102127.0           |
|Column     |balance    |Entropy            |7.789099488898172  |
+-----------+-----------+-------------------+-------------------+



/content/spark-3.3.2-bin-hadoop2/python/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [ ]:
from pydeequ.repository import FileSystemMetricsRepository, ResultKey
from pydeequ.analyzers import AnalysisRunner, ApproxCountDistinct



metrics_file = FileSystemMetricsRepository.helper_metrics_file(spark, 'metrics.json')
repository = FileSystemMetricsRepository(spark, metrics_file)

We can generate a tag per each analysis, with a perspective of time, which one was the first.

In [ ]:
key_tags = {'tag': 'age'}
resultKey = ResultKey(spark, ResultKey.current_milli_time(), key_tags)
analysisResult = AnalysisRunner(spark) \
                         .onData(df) \
                         .addAnalyzer(ApproxCountDistinct('age')) \
                         .useRepository(repository) \
                         .saveOrAppendResult(resultKey) \
                         .run()

In [ ]:
resultKey2 = ResultKey(spark, ResultKey.current_milli_time(), key_tags)

In [ ]:
analysisResult = AnalysisRunner(spark) \
                         .onData(df) \
                         .addAnalyzer(Maximum('age')) \
                         .useRepository(repository) \
                         .saveOrAppendResult(resultKey2) \
                         .run()

In [ ]:
analysisResult_metRep = repository.load() \
                           .before(ResultKey.current_milli_time()) \
                           .getSuccessMetricsAsDataFrame()
analysisResult_metRep.show()

+------+--------+-------------------+-----+-------------+---+
|entity|instance|               name|value| dataset_date|tag|
+------+--------+-------------------+-----+-------------+---+
|Column|     age|ApproxCountDistinct| 76.0|1686066659919|age|
|Column|     age|            Maximum| 95.0|1686066663723|age|
+------+--------+-------------------+-----+-------------+---+



We can also creare profiles for ech feature and save the them in a dataframe. Like is said during the classes, the ultimte goal is to have a database where we can save all this relevant information

In [ ]:
from pydeequ.profiles import ColumnProfilerRunner
result = ColumnProfilerRunner(spark) \
                .onData(df) \
                .run()

In [ ]:
list_elements = []
for col, profile in result.profiles.items():
       print(f'Column \'{col}\'')
       print('\t',f'completeness: {profile.completeness}')
       print('\t',f'approximate number of distinct values: {profile.approximateNumDistinctValues}')
       print('\t',f'datatype: {profile.dataType}')
       list_elements.append((col, profile.completeness, profile.approximateNumDistinctValues, profile.dataType))


Column 'duration'
	 completeness: 1.0
	 approximate number of distinct values: 1605
	 datatype: Integral
Column 'housing'
	 completeness: 1.0
	 approximate number of distinct values: 2
	 datatype: String
Column 'y'
	 completeness: 1.0
	 approximate number of distinct values: 2
	 datatype: String
Column 'previous'
	 completeness: 1.0
	 approximate number of distinct values: 42
	 datatype: Integral
Column 'campaign'
	 completeness: 1.0
	 approximate number of distinct values: 47
	 datatype: Integral
Column 'job'
	 completeness: 1.0
	 approximate number of distinct values: 11
	 datatype: String
Column 'balance'
	 completeness: 1.0
	 approximate number of distinct values: 7375
	 datatype: Integral
Column 'contact'
	 completeness: 1.0
	 approximate number of distinct values: 3
	 datatype: String
Column 'age'
	 completeness: 1.0
	 approximate number of distinct values: 76
	 datatype: Integral
Column 'loan'
	 completeness: 1.0
	 approximate number of distinct values: 2
	 datatype: String
Colu

In [ ]:
list_elements

[('duration', 1.0, 1605, 'Integral'),
 ('housing', 1.0, 2, 'String'),
 ('y', 1.0, 2, 'String'),
 ('previous', 1.0, 42, 'Integral'),
 ('campaign', 1.0, 47, 'Integral'),
 ('job', 1.0, 11, 'String'),
 ('balance', 1.0, 7375, 'Integral'),
 ('contact', 1.0, 3, 'String'),
 ('age', 1.0, 76, 'Integral'),
 ('loan', 1.0, 2, 'String'),
 ('education', 1.0, 4, 'String'),
 ('marital', 1.0, 3, 'String'),
 ('default', 1.0, 2, 'String'),
 ('pdays', 1.0, 553, 'Integral'),
 ('day', 1.0, 32, 'Integral'),
 ('poutcome', 1.0, 4, 'String'),
 ('month', 1.0, 12, 'String')]

In [ ]:
import pandas as pd

In [ ]:
df_profile = pd.DataFrame(list_elements, columns=('Feature', 'Completeness', 'Distinct_values', 'DataType'))

In [ ]:
df_profile

,Feature,Completeness,Distinct_values,DataType
0,duration,1.0,1605,Integral
1,housing,1.0,2,String
2,y,1.0,2,String
3,previous,1.0,42,Integral
4,campaign,1.0,47,Integral
5,job,1.0,11,String
6,balance,1.0,7375,Integral
7,contact,1.0,3,String
8,age,1.0,76,Integral
9,loan,1.0,2,String


Defining and running tests for data. This is the important step that shoud be present in our ingestion process, before starting modelling!


In [ ]:
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Warning, "Bank Check")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check.hasSize(lambda x: x >= 40000) \
        .hasMin("balance", lambda x: x == 0.0) \
        .hasMax("balance", lambda x: x == 20000)  \
        .isComplete("education")  \
        .isUnique("y")  \
        .isComplete("y")  \
        .isContainedIn("marital", ["single", "married", "divorced"]) \
        .isNonNegative("day")) \
    .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(20,False)

Python Callback server started!
+----------+-----------+------------+-----------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+---------------------------------------------------------+
|check     |check_level|check_status|constraint                                                                                                                                           |constraint_status|constraint_message                                       |
+----------+-----------+------------+-----------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+---------------------------------------------------------+
|Bank Check|Warning    |Warning     |SizeConstraint(Size(None))                                                                                                                 

/content/spark-3.3.2-bin-hadoop2/python/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


Pydeequ also has the ability to provide us suggestions for constraining our features.

In [ ]:
from pydeequ.suggestions import *

suggestionResult = ConstraintSuggestionRunner(spark)\
             .onData(df)\
             .addConstraintRule(CompleteIfCompleteRule())\
             .addConstraintRule(NonNegativeNumbersRule())\
             .addConstraintRule(RetainCompletenessRule())\
             .addConstraintRule(RetainTypeRule())\
             .addConstraintRule(UniqueIfApproximatelyUniqueRule())\
             .run()

# Constraint Suggestions in JSON format
print(json.dumps(suggestionResult, indent=2))

{
  "constraint_suggestions": [
    {
      "constraint_name": "CompletenessConstraint(Completeness(duration,None))",
      "column_name": "duration",
      "current_value": "Completeness: 1.0",
      "description": "'duration' is not null",
      "suggesting_rule": "CompleteIfCompleteRule()",
      "rule_description": "If a column is complete in the sample, we suggest a NOT NULL constraint",
      "code_for_constraint": ".isComplete(\"duration\")"
    },
    {
      "constraint_name": "ComplianceConstraint(Compliance('duration' has no negative values,duration >= 0,None))",
      "column_name": "duration",
      "current_value": "Minimum: 0.0",
      "description": "'duration' has no negative values",
      "suggesting_rule": "NonNegativeNumbersRule()",
      "rule_description": "If we see only non-negative numbers in a column, we suggest a corresponding constraint",
      "code_for_constraint": ".isNonNegative(\"duration\")"
    },
    {
      "constraint_name": "CompletenessConstrain

We can create a more friendly format to use it, afterwords, just only need to create a function customized for that effect.

In [ ]:
for suggestion in suggestionResult['constraint_suggestions']:
  print("-----")
  print(f"Suggested constraint for \'{suggestion['column_name']}\': {suggestion['description']}")
  print(f"The description for this rule is: \'{suggestion['rule_description']}\'")
  print(f"The corresponding Python code is: `{suggestion['code_for_constraint']}`")


pydeequ_validation_string = ""

# Building string from suggestions
for suggestion in suggestionResult['constraint_suggestions']:
  pydeequ_validation_string = pydeequ_validation_string + suggestion["code_for_constraint"]

# Printing string validation string
# If desired, edit this string to control what data quality validations are performed
print(pydeequ_validation_string)

-----
Suggested constraint for 'duration': 'duration' is not null
The description for this rule is: 'If a column is complete in the sample, we suggest a NOT NULL constraint'
The corresponding Python code is: `.isComplete("duration")`
-----
Suggested constraint for 'duration': 'duration' has no negative values
The description for this rule is: 'If we see only non-negative numbers in a column, we suggest a corresponding constraint'
The corresponding Python code is: `.isNonNegative("duration")`
-----
Suggested constraint for 'housing': 'housing' is not null
The description for this rule is: 'If a column is complete in the sample, we suggest a NOT NULL constraint'
The corresponding Python code is: `.isComplete("housing")`
-----
Suggested constraint for 'y': 'y' is not null
The description for this rule is: 'If a column is complete in the sample, we suggest a NOT NULL constraint'
The corresponding Python code is: `.isComplete("y")`
-----
Suggested constraint for 'previous': 'previous' is no

Create the customized function

In [ ]:
import logging

from pydeequ.checks import Check, CheckLevel, ConstrainableDataTypes
from pydeequ.verification import VerificationResult, VerificationSuite
from pyspark.sql import functions as F

logger = logging.getLogger(__name__)

def perform_checks(df, pydeequ_validation_string):
    """Perform and log data quality checks."""

    # Initializing
    check = \
        Check(spark_session=spark,
              level=CheckLevel.Warning,
              description="Data Quality Check")

    # Building validation string of constraints to check
    pydeequ_validation_string_to_check = "check" + pydeequ_validation_string

    # Checking constraints
    checked_constraints = \
        (VerificationSuite(spark)
         .onData(df)
         .addCheck(eval(pydeequ_validation_string_to_check))
         .run())

    # Returning results as DataFrame
    df_checked_constraints = \
        (VerificationResult
         .checkResultsAsDataFrame(spark, checked_constraints))

    logger.info(
        df_checked_constraints.show(n=df_checked_constraints.count(),
                                    truncate=False)
    )

    # Filtering for any failed data quality constraints
    df_checked_constraints_failures = \
        (df_checked_constraints
         .filter(F.col("constraint_status") == "Failure"))

    # If any data quality check fails, raise exception
    if df_checked_constraints_failures.count() > 0:
        logger.info(df_checked_constraints_failures\
              .show(n=df_checked_constraints_failures.count(), truncate=False))

In [ ]:
pydeequ_validation_string

'.isComplete("duration").isNonNegative("duration").isComplete("housing").isComplete("y").isComplete("previous").isNonNegative("previous").isComplete("campaign").isNonNegative("campaign").isComplete("job").isComplete("balance").isComplete("contact").isComplete("age").isNonNegative("age").isComplete("loan").isComplete("education").isComplete("marital").isComplete("default").isComplete("pdays").isComplete("day").isNonNegative("day").isComplete("poutcome").isComplete("month")'

In [ ]:
perform_checks(df, pydeequ_validation_string)

/content/spark-3.3.2-bin-hadoop2/python/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+------------------+-----------+------------+--------------------------------------------------------------------------------------------------------------------+-----------------+------------------+
|check             |check_level|check_status|constraint                                                                                                          |constraint_status|constraint_message|
+------------------+-----------+------------+--------------------------------------------------------------------------------------------------------------------+-----------------+------------------+
|Data Quality Check|Warning    |Success     |CompletenessConstraint(Completeness(duration,None))                                                                 |Success          |                  |
|Data Quality Check|Warning    |Success     |ComplianceConstraint(Compliance(duration is non-negative,COALESCE(CAST(duration AS DECIMAL(20,10)), 0.0) >= 0,None))|Success          |                  |
